In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("vectorbt_reports/expanding_master_backtest_stats.csv")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=['Sharpe Ratio'])
df = df.drop(columns=['Start Value', 'End Value'])
#df = df[df["asset"]=="NVDA"]
#display(df[df["asset"]=="NVDA"])

In [6]:
returns = df['Total Return [%]'] / 100
cumulative_return = (1 + returns).prod() - 1
N = len(returns)  # number of periods
annualized_return = (1 + cumulative_return)**(12 / N) - 1
annualized_volatility = returns.std() * np.sqrt(12)
sharpe_annual = annualized_return / annualized_volatility
print(sharpe_annual)

-0.49514280691703727


In [7]:
def compute_annualized_sharpe(df):
    results = []

    for asset, group in df.groupby("asset"):
        # Convert returns from % to decimals
        returns = group['Total Return [%]'] / 100
        benchmark_returns = group['Benchmark Return [%]'] / 100
        N = len(returns)

        if N < 2:
            continue  # Skip assets with insufficient data

        # Strategy Sharpe
        cumulative_return = (1 + returns).prod() - 1
        annualized_return = (1 + cumulative_return)**(12 / N) - 1
        annualized_volatility = returns.std() * np.sqrt(12)
        sharpe_strategy = annualized_return / annualized_volatility if annualized_volatility > 0 else np.nan

        # Benchmark Sharpe
        cumulative_bench = (1 + benchmark_returns).prod() - 1
        annualized_bench_return = (1 + cumulative_bench)**(12 / N) - 1
        annualized_bench_vol = benchmark_returns.std() * np.sqrt(12)
        sharpe_benchmark = annualized_bench_return / annualized_bench_vol if annualized_bench_vol > 0 else np.nan

        results.append({
            'asset': asset,
            'strategy_sharpe': sharpe_strategy,
            'benchmark_sharpe': sharpe_benchmark
        })

    return pd.DataFrame(results)

In [8]:
sharpe_df = compute_annualized_sharpe(df)
display(sharpe_df.sort_values(by='strategy_sharpe', ascending=False))

,asset,strategy_sharpe,benchmark_sharpe
14,TSLA,0.330148,0.965867
3,BTC=F,0.317374,0.580419
2,BTC-USD,-0.335988,0.654541
13,NVDA,-0.487816,-0.231651
9,GC=F,-0.490583,2.864663
0,AAPL,-0.909311,0.652176
10,GOOGL,-0.913706,-0.416418
8,ES=F,-0.982081,0.789796
5,CL=F,-1.087253,-0.187782
1,AMZN,-1.504890,0.382994


In [13]:
df = pd.read_csv("vectorbt_reports/fixed_master_backtest_stats.csv")
display(df)


,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
0,2024-04-01,2025-04-01,252 days,10000.0,13137.165507,31.371655,31.265073,100.0,542.728558,6.264875,...,-2.165967,8 days 17:35:59.999999999,2 days 21:00:00,2.503729,136.398500,1.957834,7.736821,1.417401,3.096980,AAPL
1,2024-04-01 00:00:00,2025-04-01 00:00:00,252 days 00:00:00,10000.0,9997.757088,-0.022429,6.188869,100.0,572.596494,15.829473,...,-2.242341,8 days 03:25:42.857142857,3 days 16:36:55.384615384,0.999284,-0.083071,0.126588,-0.002052,1.021224,0.180745,AMZN
2,2024-04-01 00:00:00,2025-04-01 00:00:00,366 days 00:00:00,10000.0,10931.810653,9.318107,22.190167,100.0,741.245637,22.302727,...,-3.062346,8 days 12:00:00,3 days 04:34:17.142857142,1.129074,28.236686,0.424560,0.416608,1.093658,0.681273,BTC-USD
3,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,13759.319700,37.593197,21.270247,100.0,544.290621,13.697510,...,-2.989688,10 days 00:00:00,2 days 20:18:27.692307692,1.837350,163.448683,1.247011,4.268794,1.259047,2.173086,BTC=F
4,2024-04-01 00:00:00,2025-03-31 00:00:00,260 days 00:00:00,10000.0,9395.990535,-6.040095,2.404336,100.0,587.625258,9.166187,...,-0.500663,8 days 00:00:00,4 days 02:05:13.043478260,0.466430,-20.133649,-1.177725,-0.913649,0.813453,-1.598499,CHFUSD=X
5,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,7901.557657,-20.984423,-14.944454,100.0,444.986551,25.912435,...,-2.488019,11 days 10:17:08.571428571,3 days 09:20:00,0.484209,-83.937694,-1.140348,-1.111743,0.818315,-1.505331,CL=F
6,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,9735.050777,-2.649492,-0.723666,100.0,453.359657,5.552026,...,-0.609411,15 days 20:00:00,2 days 16:56:28.235294117,0.741010,-11.519531,-0.634500,-0.684410,0.895385,-0.839334,DX-Y_NYB
7,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,9590.567455,-4.094325,-0.777941,100.0,428.744900,6.231313,...,-0.598515,17 days 09:36:00,2 days 16:56:28.235294117,0.588204,-18.610570,-1.002544,-0.939270,0.835274,-1.280666,DX=F
8,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,10738.973865,7.389739,7.162079,100.0,474.492750,8.153628,...,-0.815456,11 days 07:12:00,4 days 12:00:00,1.695301,33.589721,0.931205,1.328628,1.173037,1.298706,ES=F
9,2024-04-01 00:00:00,2025-04-01 00:00:00,253 days 00:00:00,10000.0,11420.651581,14.206516,39.454500,100.0,675.858793,6.641395,...,-1.003820,7 days 19:12:00,1 days 22:30:00,1.810387,45.827470,1.399073,3.180645,1.268683,1.952890,GC=F
